### Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ocslc.switched_linear_mpc import SwitchedLinearMPC

In [ ]:
from cycler import cycler

default_cycler = (
    cycler(color=[
        '#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30',
        '#4DBEEE', '#A2142F', '#FF6F00', '#8DFF33', '#33FFF7',
    ]) +
    cycler('linestyle', [
        '-', '--', '-.', ':', '-',
        '--', '-.', ':', '-', '--'
    ])
)

textsize = 16
labelsize = 18

plt.rc('font', family='serif', serif='Times')
plt.rcParams["text.usetex"] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{amsfonts} \DeclareMathAlphabet{\mathcal}{OMS}{cmsy}{m}{n}')
plt.rc('xtick', labelsize=textsize)
plt.rc('ytick', labelsize=textsize)
plt.rc('axes', titlesize=labelsize, labelsize=labelsize, prop_cycle=default_cycler)
plt.rc('legend', fontsize=textsize)
plt.rc('grid', linestyle='-.', alpha=0.5)
plt.rc('axes', grid=True)

plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
def solve_pannocchia(
    n_phases,
    multiple_shooting=True,
    integration_method='int',
    hybrid=False,
):
    model = {
        'A': [np.array([[-0.1, 0, 0], [0, -2, -6.25], [0, 4, 0]])],
        'B': [np.array([[0.25], [2], [0]])],
    }

    n_states = model['A'][0].shape[0]
    n_inputs = model['B'][0].shape[1]

    time_horizon = 10

    x0 = np.array([1.3440, -4.5850, 5.6470])

    swi_lin_mpc = SwitchedLinearMPC(
        model, n_phases, time_horizon, auto=False,
        multiple_shooting=multiple_shooting, x0=x0, propagation=integration_method,
        hybrid=hybrid
    )

    Q = 1. * np.eye(n_states)
    R = 0.1 * np.eye(n_inputs)
    E = 0. * np.eye(n_states)

    swi_lin_mpc.precompute_matrices(x0, Q, R, E)

    states_lb = np.array([-100, -100, -100])
    states_ub = np.array([100, 100, 100]) 

    swi_lin_mpc.set_bounds(-1, 1, states_lb, states_ub)

    if swi_lin_mpc.multiple_shooting:
        swi_lin_mpc.multiple_shooting_constraints(x0)

    swi_lin_mpc.set_cost_function(Q, R, x0)

    # Set the initial guess  
    swi_lin_mpc.set_initial_guess(time_horizon, x0)

    swi_lin_mpc.create_solver('ipopt', print_level=0, print_time=False)

    inputs_opt, deltas_opt, _ = swi_lin_mpc.solve()

    # swi_lin_mpc.plot_optimal_solution(deltas_opt, inputs_opt)
    
    return inputs_opt, deltas_opt

### Generate Data

In [ ]:
data = []

for multiple_shooting_flag in [True, False]:
    for integration_method in ['int', 'exp']:
        for hybrid in [True, False]:
            if not multiple_shooting_flag and hybrid:
                continue
            
            shooting_method = "MS" if multiple_shooting_flag else "SS"
            inputs_opt, deltas_opt = solve_pannocchia(
                80, multiple_shooting=multiple_shooting_flag, integration_method=integration_method,
                hybrid=hybrid,
            )
            data.append([
                shooting_method, integration_method, hybrid,
                inputs_opt.copy(), deltas_opt.copy(),
            ])

### Plot Histograms

In [ ]:
column_names = ['Shooting Method', 'Integration Method', 'Hybrid', 'Inputs', 'Deltas']
df = pd.DataFrame(data, columns=column_names)

fig, axs = plt.subplots(1, 4, figsize=(12.8, 3.6))

for i, integration_method in enumerate(['Int', 'Exp']):
    for j, multiple_shooting_flag in enumerate([False, True]):
        shooting_method = "MS" if multiple_shooting_flag else "SS"
        delta = df[
            (df['Shooting Method'] == shooting_method) &
            (df['Integration Method'] == integration_method.lower()) & 
            (df['Hybrid'] == False)
        ]['Deltas']

        axs[2*i+j].hist(delta, bins='auto', edgecolor='black', alpha=0.7)
        axs[2*i+j].set_xlabel('Timestep Length')
        axs[2*i+j].text(
            0.5, 0.95,
            f'{shooting_method} - {integration_method}',
            transform=axs[2*i+j].transAxes,
            fontsize=textsize,
            va='top', ha='center',
        )
        axs[2*i+j].grid(True)
axs[0].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('./hist.pdf', format='pdf', bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6.4, 3.6))

for i, integration_method in enumerate(['Int', 'Exp']):
    delta = df[
        (df['Integration Method'] == integration_method.lower()) & 
        (df['Hybrid'] == True)
    ]['Deltas']

    axs[i].hist(delta, bins='auto', edgecolor='black', alpha=0.7)
    axs[i].set_xlabel('Timestep Length')
    axs[i].text(
        0.5, 0.95,
        f'Hybrid - {integration_method}',
        transform=axs[i].transAxes,
        fontsize=textsize,
        va='top', ha='center',
    )
    axs[i].grid(True)
axs[0].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('./hist_2.pdf', format='pdf', bbox_inches='tight')

### Plot Inputs

In [ ]:
column_names = ['Shooting Method', 'Integration Method', 'Hybrid', 'Inputs', 'Deltas']
df = pd.DataFrame(data, columns=column_names)

fig, axs = plt.subplots(
    2, 2,
    sharex=True, sharey=True, figsize=(9.6, 7.2),
    gridspec_kw={'wspace': 0.075, 'hspace': 0.075}
)

for i, integration_method in enumerate(['Int', 'Exp']):
    for j, multiple_shooting_flag in enumerate([False, True]):
        shooting_method = "MS" if multiple_shooting_flag else "SS"
        input = df[
            (df['Shooting Method'] == shooting_method) &
            (df['Integration Method'] == integration_method.lower()) & 
            (df['Hybrid'] == False)
        ]['Inputs']
        
        delta = df[
            (df['Shooting Method'] == shooting_method) &
            (df['Integration Method'] == integration_method.lower()) & 
            (df['Hybrid'] == False)
        ]['Deltas']

        axs[i,j].step(
            np.cumsum(delta.values.tolist()), np.concatenate(input.values.tolist()), where='pre')
        axs[i,j].text(
            0.5, 0.95,
            f'{shooting_method} - {integration_method}',
            transform=axs[i,j].transAxes,
            fontsize=textsize,
            va='top', ha='center',
        )
        axs[i,j].grid(True)
        axs[i,j].set_xlim([0, 10])
        
axs[1,0].set_xlabel('Time')
axs[1,1].set_xlabel('Time')
axs[0,0].set_ylabel('Input')
axs[1,0].set_ylabel('Input')

plt.tight_layout()
plt.savefig('./input_pann.pdf', format='pdf', bbox_inches='tight')